# SFT
[SFT与pretrain阶段在dataset上的不同](http://www.chinasem.cn/article/361510)<br>
简而言之，最大的不同就是assistant回答的部分计算loss,而user的部分不计算loss了

## SFT单卡代码
这里我们参考pretrain阶段的代码来看即可

#### 训练参数

In [ ]:
import torch
class sft_args:
    out_dir = "../out"
    epochs = 1
    batch_size = 32
    learning_rate = 5e-4
    device = "cuda:3" if torch.cuda.is_available() else "cpu"
    dtype = "bfloat16"
    use_wandb = False
    wandb_project = "MiniMind-SFT-512"
    num_workers = 1
    ddp = False
    accumulation_steps = 8
    grad_clip = 1.0
    warmup_iters = 0
    log_interval = 100
    save_interval = 100
    local_rank = -1
    embed_dim = 512
    block_num = 8
    max_seqlen = 1024
    use_moe = False
    data_path = "../data/sft_data.jsonl"  # toy_dataset  
    # data_path = "../data/sft_512.jsonl" #full_dataset

#### 加载model

In [ ]:
import sys
import os

# 获取当前 notebook 所在目录（trainer/）
current_dir = os.path.dirname(os.path.abspath("__file__"))  # 注意 Jupyter 中可能需要调整
# 或者直接写死路径
current_dir = "/data/zyp/jinbu/ZZY/minimind-v-learn/trainer"

# 上一级目录就是项目根目录，拼接 model 路径
model_dir = os.path.join(os.path.dirname(current_dir), "model")
sys.path.append(model_dir)

# 现在可以用绝对导入
from model import MinimindForCausalLM, MinimindConfig
train_args = sft_args()
train_args.save_dir = os.path.join(train_args.out_dir)
# 确保输出目录存在
os.makedirs(train_args.save_dir, exist_ok=True)
# 初始化模型配置
config = MinimindConfig(
    embed_dim=train_args.embed_dim,
    block_num=train_args.block_num,
    max_seqlen=train_args.max_seqlen,
)
print(f'查看工作设备 {train_args.device}')

/data/zyp/miniconda3/envs/minimind/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


查看工作设备 cuda:3


#### 单卡训练

In [3]:
from transformers import AutoTokenizer
import math
from torch.utils.data import DataLoader
import sys
from pathlib import Path

# 项目根目录：/data/zyp/jinbu/ZZY/minimind-v-learn
root_dir = Path("/data/zyp/jinbu/ZZY/minimind-v-learn")

# 将根目录添加到 Python 可搜索路径
sys.path.append(str(root_dir))
from dataset.lm_dataset import PretrainDataset,SFTDataset

def Logger(content):
    print(content)

def init_model(lm_config):
    tokenizer = AutoTokenizer.from_pretrained('../model/')
    model = MinimindForCausalLM(lm_config).to(train_args.device)
    moe_path = '_moe' if train_args.use_moe else ''
    ckp = f'{train_args.save_dir}/pretrain_{lm_config.embed_dim}{moe_path}.pth'
    state_dict = torch.load(ckp, map_location=train_args.device)
    model.load_state_dict(state_dict, strict=False)
    Logger(f'加载模型参数 {ckp}')
    Logger(f'LLM可训练总参数量：{sum(p.numel() for p in model.parameters() if p.requires_grad) / 1e6:.3f} 百万')
    return model, tokenizer


model, tokenizer = init_model(config)
train_ds = SFTDataset(
    data_path=train_args.data_path,
    tokenizer=tokenizer,
    max_seqlen=train_args.max_seqlen,
)   
train_loader = DataLoader(
    train_ds,
    batch_size=train_args.batch_size,
    shuffle=False,
    num_workers=train_args.num_workers,
    pin_memory=True,
    drop_last=False
)

loader = iter(train_loader)
(X,Y, loss_mask) = next(loader)
# print(f'打印一个 batch 的数据:\nX: {X}\nY: {Y}\nloss_mask: {loss_mask}\n')
# print(f'打印一个 iter 的数据:\n{next(loader)}\n')
# print(f'数据集大小：{len(train_ds)}, DataLoader 大小：{len(loader)}')
print(f"X.tolist()= {X.tolist()}")

加载模型参数 ../out/pretrain_512.pth
LLM可训练总参数量：38.075 百万
bos_id=[2, 77, 95, 95, 85, 315, 3932, 96], eos_id=[1]


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


res=<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
你好吗？<|im_end|>
<|im_start|>assistant
我很好，谢谢！你呢？<|im_end|>
<|im_start|>user
我也很好，谢谢！<|im_end|>
<|im_start|>assistant
太好了！祝你今天愉快！<|im_end|>



/data/zyp/jinbu/ZZY/minimind-v-learn/dataset/lm_dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long)
/data/zyp/jinbu/ZZY/minimind-v-learn/dataset/lm_dataset.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long)


res=<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
你喜欢什么运动？<|im_end|>
<|im_start|>assistant
我喜欢跑步和游泳。你呢？<|im_end|>
<|im_start|>user
我喜欢打篮球！<|im_end|>
<|im_start|>assistant
篮球很棒！是一个很好的团队运动。<|im_end|>

X.tolist()= [[2, 95, 101, 315, 81, 89, 211, 69, 91, 97, 233, 313, 81, 233, 77, 233, 84, 81, 88, 92, 82, 97, 88, 233, 77, 95, 95, 85, 315, 3932, 96, 1, 211, 2, 97, 95, 3717, 211, 2899, 926, 433, 1, 211, 2, 77, 95, 95, 85, 315, 3932, 96, 211, 309, 2048, 273, 2088, 1005, 405, 1357, 433, 1, 211, 2, 97, 95, 3717, 211, 4425, 2048, 273, 2088, 1005, 1, 211, 2, 77, 95, 95, 85, 315, 3932, 96, 211, 840, 4780, 1005, 2670, 405, 1279, 3462, 1005, 1, 211, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3

### 优化器和混合精度设置

In [4]:
# 优化器方面 选择 AdamW 优化器 并在混精度场景下创建 scaler 进行梯度缩放避免数值下溢
from torch import optim
from contextlib import nullcontext
scaler = torch.cuda.amp.GradScaler(enabled=(train_args.dtype in ['float16', 'bfloat16']))
optimizer = optim.AdamW(model.parameters(), lr=train_args.learning_rate)

device_type = "cuda" if "cuda" in train_args.device else "cpu"
ctx = nullcontext() if device_type == "cpu" else torch.cuda.amp.autocast() # 在 cuda 上启动混精度训练，否则空白上下文

### 设置训练函数

In [5]:
from torch import nn
import time
iter_per_epoch = len(train_loader) # 计算每个 epoch 的迭代次数
def get_lr(current_step, total_steps, lr):
    # 余弦退火学习率调度
    return lr / 10 + 0.5 * lr * (1 + math.cos(math.pi * current_step / total_steps))

def train_epoch(epoch):
    loss_fct = nn.CrossEntropyLoss(reduction='none')
    start_time = time.time()
    for step, (X, Y, loss_mask) in enumerate(train_loader):
        X = X.to(train_args.device)
        Y = Y.to(train_args.device)
        loss_mask = loss_mask.to(train_args.device)
        print(f"loss_mask = {loss_mask}")
        lr = get_lr(epoch * iter_per_epoch + step, train_args.epochs * iter_per_epoch, train_args.learning_rate)
        for param_group in optimizer.param_groups:
            param_group['lr'] = lr

        with ctx:
            res = model(X)
            # print(f"res={res}")
            # print(f"X = {X}")
            if torch.isnan(res.logits).any() or torch.isinf(res.logits).any():
                Logger(f"Warning: logits contains NaN/Inf at step {step}")
                # 打印logits的范围，辅助排查
                Logger(f"logits range: {res.logits.min().item()} ~ {res.logits.max().item()}")
            loss = loss_fct(
                res.logits.view(-1, res.logits.size(-1)),
                Y.view(-1)
            ).view(Y.size())
            print(f"loss_mask.sum(): {loss_mask.sum()}")
            loss = (loss * loss_mask).sum() / loss_mask.sum() # 这里的loss 是有效非pad的token的平均loss
            # loss += res.aux_loss
            loss = loss / train_args.accumulation_steps

        scaler.scale(loss).backward()

        if (step + 1) % train_args.accumulation_steps == 0:
            scaler.unscale_(optimizer)
            torch.nn.utils.clip_grad_norm_(model.parameters(), train_args.grad_clip)

            scaler.step(optimizer)
            scaler.update()

            optimizer.zero_grad(set_to_none=True)  # 清空梯度，为下一个iter做准备

        if step % train_args.log_interval == 0:
            spend_time = time.time() - start_time
            Logger(
                'Epoch:[{}/{}]({}/{}) loss:{:.3f} lr:{:.12f} epoch_Time:{}min:'.format(
                    epoch + 1,
                    train_args.epochs,
                    step,
                    iter_per_epoch,
                    loss.item() * train_args.accumulation_steps,
                    optimizer.param_groups[-1]['lr'],
                    spend_time / (step + 1) * iter_per_epoch // 60 - spend_time // 60))


        if (step + 1) % train_args.save_interval == 0:
            model.eval()
            moe_path = '_moe' if train_args.use_moe else ''
            ckp = f'{train_args.save_dir}/sft_full_{config.embed_dim}{moe_path}.pth'
            Logger(f'保存模型到 {ckp}')
            state_dict = model.state_dict()

            state_dict = {k: v.half() for k, v in state_dict.items()}  # 半精度保存
            torch.save(state_dict, ckp)
            model.train()

### 开始训练

In [6]:
for epoch in range(train_args.epochs):
    train_epoch(epoch)

res=<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
你好吗？<|im_end|>
<|im_start|>assistant
我很好，谢谢！你呢？<|im_end|>
<|im_start|>user
我也很好，谢谢！<|im_end|>
<|im_start|>assistant
太好了！祝你今天愉快！<|im_end|>



/data/zyp/jinbu/ZZY/minimind-v-learn/dataset/lm_dataset.py:104: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  X = torch.tensor(input_ids[:-1], dtype=torch.long)
/data/zyp/jinbu/ZZY/minimind-v-learn/dataset/lm_dataset.py:105: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  Y = torch.tensor(input_ids[1:], dtype=torch.long)


res=<|im_start|>system
You are a helpful assistant<|im_end|>
<|im_start|>user
你喜欢什么运动？<|im_end|>
<|im_start|>assistant
我喜欢跑步和游泳。你呢？<|im_end|>
<|im_start|>user
我喜欢打篮球！<|im_end|>
<|im_start|>assistant
篮球很棒！是一个很好的团队运动。<|im_end|>



huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


loss_mask = tensor([[0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]], device='cuda:3')
loss_mask.sum(): 40
Epoch:[1/1](0/1) loss:4.546 lr:0.000550000000 epoch_Time:0.0min:


### BUGs
loss_mask出现了.sum为0的问题,经过排查发现是因为
```
    def _generate_loss_mask(self, input_ids):
        loss_mask = [0] * len(input_ids)
        i = 0
        while i < len(input_ids):
            if input_ids[i:i + len(self.bos_id)] == self.bos_id:
                start = i + len(self.bos_id)
                end = start
                while end < len(input_ids):
                    if input_ids[end:end + len(self.eos_id)] == self.eos_id:
                        break
                    end += 1
                for j in range(start + 1, min(end + len(self.eos_id) + 1, self.max_length)):
                    loss_mask[j] = 1
                i = end + len(self.eos_id) if end < len(input_ids) else len(input_ids)
            else:
                i += 1
        return loss_mask
```
函数中间的`if input_ids[i:i + len(self.bos_id)] == self.bos_id:`  有问题，input_ids作为Tensor无法作为list进行一一比对，所以不能这么操作<br>
只好修改了input_ids比对过程，转为List来进行一一比对，随后便正常了